In [12]:
import pandas as pd
import numpy as np
import quandl
from datetime import datetime, timedelta
import json
import matplotlib.pyplot as plt


from dateutil.parser import parse
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss, precision_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score, f1_score, make_scorer, recall_score, average_precision_score

from sklearn.utils.multiclass import unique_labels
from sklearn.utils.fixes import signature
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from function_library import write_merged_frames, stack_frames, create_labels, clean_features
from function_library import encode_sectors, prepare_partitions
from function_library import create_hard_classes

In [13]:
# create list of 'surprise' files

surp_files = (['surp_1q14.csv',
               'surp_2q14.csv',
               'surp_3q14.csv',
               'surp_4q14.csv',
               'surp_1q15.csv',
               'surp_2q15.csv',
               'surp_3q15.csv',
               'surp_4q15.csv',
               'surp_1q16.csv',
               'surp_2q16.csv',
               'surp_3q16.csv',
               'surp_4q16.csv',
               'surp_1q17.csv',
               'surp_2q17.csv',
               'surp_3q17.csv',
               'surp_4q17.csv',
               'surp_1q18.csv',
               'surp_2q18.csv',
               'surp_3q18.csv'])

In [14]:
# create list of 'features' files

features_files = (['features_1q14.csv',
                   'features_2q14.csv',
                   'features_3q14.csv',
                   'features_4q14.csv',
                   'features_1q15.csv',
                   'features_2q15.csv',
                   'features_3q15.csv',
                   'features_4q15.csv',
                   'features_1q16.csv',
                   'features_2q16.csv',
                   'features_3q16.csv',
                   'features_4q16.csv',
                   'features_1q17.csv',
                   'features_2q17.csv',
                   'features_3q17.csv',
                   'features_4q17.csv',
                   'features_1q18.csv',
                   'features_2q18.csv',
                   'features_3q18.csv'])


In [15]:
# data pipeline script

combined_frames = write_merged_frames(surp_files, features_files)
combined_full = stack_frames(combined_frames)
create_labels('combined_full_set')
clean_features('combined_full_set')
encode_sectors('combined_clean')
X_train, X_test, X_oos, y_train, y_test, y_oos = prepare_partitions('combined_clean')